# Download the dataset
![](https://github.com/phelber/EuroSAT/blob/master/eurosat_overview_small.jpg?raw=true)


In [ ]:
!wget https://madm.dfki.de/files/sentinel/EuroSAT.zip --no-check-certificate
!unzip -q /content/EuroSAT.zip
!rm -r EuroSAT
!mv 2750 EuroSAT
!ls -l /content/EuroSAT

# Cloning repository with base code

In [ ]:
import sys
!rm -r /content/QML-tutorial
!git clone https://github.com/alessandrosebastianelli/QML-tutorial.git
if '/content/QML-tutorial' not in sys.path: sys.path.append('/content/QML-tutorial')

## Installing Missing packages

In [ ]:
!pip -q install qiskit==0.23.0

# EuroSAT classification with Hybrid Quantum Neural Networks

In [ ]:
from utils.DatasetHandler import DatasetHandler
from qc.QiskitCircuit import QiskitCircuit
from models.HybridNet import HybridNet
from utils.utils import print_CF

from config import *

from sklearn.metrics import confusion_matrix, classification_report
import torch.optim as optim
import torch.nn as nn
import torch
import os

# Suppressing warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_root = '/content/EuroSAT'

print('Loading Dataset')
dh = DatasetHandler(dataset_root)

classes = []
for i, c in enumerate(dh.classes):
    cl = c.split(os.path.sep)[-1]
    classes.append(cl)
classes.sort()

print('[*] Classes: ', classes)
imgs, labels = dh.load_paths_labels(dataset_root, classes=classes)

print('[*] Size: ', len(imgs))


tra_imgs, tra_lbls, val_imgs, val_lbls = dh.train_validation_split(imgs, labels, 0.2)
print('[*] Training Size: ', len(tra_imgs))
print('[*] Validation Size: ', len(val_imgs))

In [ ]:
print('Initialize Quantum Hybrid Neural Network')
network = HybridNet()
optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)

print('Printing Quantum Circuit')
circuit = QiskitCircuit(NUM_QUBITS, SIMULATOR, NUM_SHOTS)
print(circuit.circuit.draw(output='text', scale=1/NUM_LAYERS))

print('Printing Quantum Circuit Parameters')
print('[*] Number of Qubits:   {}'.format(NUM_QUBITS))
print('[*] Number of R Layers: {}'.format(NUM_LAYERS))
print('[*] Number of Outputs:  {}'.format(NUM_QC_OUTPUTS))
print('[*] Number of Shots:    {}'.format(NUM_SHOTS))

In [ ]:
train_loader = iter(dh.data_loader(tra_imgs, tra_lbls, batch_size=1, img_shape=(64,64,3)))
test_loader  = iter(dh.data_loader(val_imgs, val_lbls, batch_size=1, img_shape=(64,64,3)))

In [ ]:
print('Training')
train_loss_list = []
val_loss_list = []
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    total_loss = []
    for batch_idx in range(len(tra_imgs)):
        data, target = next(train_loader)
        optimizer.zero_grad()
        # Forward pass
        output = network(data)
        # Calculating loss
        loss = loss_func(output, target)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        total_loss.append(loss.item())

        print('\r\t\t [*] [Epoch %d/%d] [Batch %d/%d] [Train Loss %f] ' % (epoch, EPOCHS, batch_idx, len(tra_imgs) - 1, loss.item()),
              end='\t\t')

    with torch.no_grad():
        val_loss = []
        targets = []
        predictions = []
        for batch_idx in range(len(val_imgs)):
            data, target = next(test_loader)
            output = network(data)
            loss = loss_func(output, target)
            val_loss.append(loss.item())
            targets.append(target.item())
            predictions.append(network.predict(data).item())

    train_loss_list.append(sum(total_loss) / len(total_loss))
    val_loss_list.append(sum(val_loss) / len(val_loss))

    print('[Val Loss %f] ' % (val_loss_list[-1]))

    if epoch % 3 == 1:
        cf = confusion_matrix(targets, predictions, normalize='true')
        cr = classification_report(targets, predictions, target_names=classes, digits=4)

        print('\t\t [*] Confusion Matrix:')
        print_CF(cf, classes)
        print('\t\t [*] Classification Report:')
        print(cr)